In [ ]:
import os
import glob
import math
import random
import gc

import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import Model, Sequential, load_model
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda, Dropout, Concatenate, Add
from tensorflow.python.keras.layers import Conv2DTranspose, Activation, Cropping2D, BatchNormalization
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.python.keras.optimizers import Adam, SGD

config = tf.ConfigProto(  
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
sess = tf.Session(config=config)


In [ ]:
NO_CLASS = 20
IGNORE_LABEL = np.uint8(19)
RESIZE_HEIGHT =512 #image height for network inputting
RESIZE_WIDTH = 1024  #image width for network inputting
#KNL_INITLZ = 'he_uniform'
KNL_INITLZ = 'he_normal'

LEARN_RATE = np.float32(1.0e-4)
BETA_1 = np.float32(0.99) # for adam
MOMENTUM= np.float32(0.9)

AL2_VALUE1 = np.float32(1.0e-3) # for encoder
AL2_VALUE2 = np.float32(1.0e-3)
AL2_VALUE3 = np.float32(1.0e-3)
AL2_VALUE4 = np.float32(1.0e-3)

FILT_SIZE = (5, 5) # for encoder
FILT_NO1 = 96
FILT_NO2 = 64
FILT_NO3 = 32
FILT_NO4 = 16

L2_VALUE1 = np.float32(5.0e-4) #for FCN
L2_VALUE2 = np.float32(5.0e-4)
L2_VALUE3 = np.float32(5.0e-4)

BATCH_SIZE = 5
EPOCHS1 = 1000
EPOCHS2 = 500


In [ ]:
path_train_image = "C:/Users/test/Documents/Cityscapes/Image/Train_Image"  #path to train image folder
path_val_image = "C:/Users/test/Documents/Cityscapes/Image/Val_Image"  #path to val image folder

path_train_grth = "C:/Users/test/Documents/Cityscapes/GtImage/Train"  #path to Train grth folder
path_val_grth = "C:/Users/test/Documents/Cityscapes/GtImage/Val" #path to Val grth folder

path_train_predict_folder = "C:/Users/test/Documents/Cityscapes/Predict/Train_Predict" #path to Predict folder
path_val_predict_folder = "C:/Users/test/Documents/Cityscapes/Predict/Val_Predict" #path to Predict folder

#checkpoint_path = "C:/Users/test/Documents/Cityscapes/Weights/cp-{epoch:04d}-{val_acc:.4f}.hdf5"  #path to checkpoint(Weights) folder
checkpoint_path = "C:/Users/test/Documents/Cityscapes/Weights/best_weights.hdf5"  #path to checkpoint(Weights) folder

encoder_path = "C:/Users/test/Documents/Cityscapes/Encoder_Weights/encoder_2022_1_28.hdf5"  #path to Encoder_Weights folder


In [ ]:
palette = np.zeros(256*3, dtype = np.uint8)
palette = np.array(palette).reshape(256, 3)
palette[0] =  np.array([128, 64,128]) #made by copying labels[ ]
palette[1] =  np.array([244, 35,232])
palette[2] =  np.array([ 70, 70, 70])
palette[3] =  np.array([102,102,156])
palette[4] =  np.array([190,153,153])
palette[5] =  np.array([153,153,153])
palette[6] =  np.array([250,170, 30])
palette[7] =  np.array([220,220,  0])
palette[8] =  np.array([107,142, 35])
palette[9] =  np.array([152,251,152])
palette[10] = np.array([ 70,130,180])
palette[11] = np.array([220, 20, 60])
palette[12] = np.array([255,  0,  0])
palette[13] = np.array([  0,  0,142])
palette[14] = np.array([  0,  0, 70])
palette[15] = np.array([  0, 60,100])
palette[16] = np.array([  0, 80,100])
palette[17] = np.array([  0,  0,230])
palette[18] = np.array([119, 11, 32])
palette[19] = np.array([  0,  0,  0])
palette = np.array(palette).reshape(-1)
palette1 = palette[0: 3*22]
palette1 = np.array(palette1).reshape(22, 3)
print("palette1 shape: ", palette1.shape)
#print("palette1: \n", palette1)


In [ ]:
def data_transform(path_image, path_grth):
    no_image = 0
    image_array_list = []
    grth_image_array_list = []
    filename = os.listdir(path_image)[0]
    image = Image.open(path_image + "/" + filename)
    width = image.width
    tg_size = (RESIZE_WIDTH, RESIZE_HEIGHT)
    for filename in os.listdir(path_image):
        no_image += 1
        image = Image.open(path_image + "/" + filename)
        image = image.convert("RGB")
        if width != RESIZE_WIDTH:
            image = image.resize(tg_size, PIL.Image.ANTIALIAS)    
        image_array = np.asarray(image, dtype=np.uint8)
        image_array_list.append(image_array)
        length = len(filename)
        length1 = length - 16
        string = filename[0:length1]
        filename1 = string + "_gtFine_labelIds.png"
        try:
            grth_image = Image.open(path_grth + "/" + filename1)
        except:
            print(filename1 + "does nor exist")
            break
        if width != RESIZE_WIDTH:
            grth_image = grth_image.resize(tg_size, PIL.Image.NEAREST)    
        grth_image_array = np.asarray(grth_image, dtype=np.uint8)
        grth_image_array_list.append(grth_image_array)
            
#        if no_image == 20:  #break for debug mode
#            break
           
    return no_image, image_array_list, grth_image_array_list

In [ ]:
train_no_image, train_image_array_list, train_grth_image_array_list = data_transform(path_train_image, path_train_grth)
print("trai_no_image = ", train_no_image)
print("train data have been prepared successfully")


In [ ]:
val_no_image, val_image_array_list, val_grth_image_array_list = data_transform(path_val_image, path_val_grth)
print("val_no_image = ", val_no_image)
print("val data have been prepared successfully")


In [ ]:
print("display processed train data for checking")

from IPython.display import display_png

no = 0
for i in range(2):
    print("processed no = ", no)
    train_image = Image.fromarray(np.uint8(train_image_array_list[i]), mode="RGB")
    display_png(train_image)
    grth_image = Image.fromarray(np.uint8(train_grth_image_array_list[i]), mode="P")
    grth_image.putpalette(palette)
    display_png(grth_image)
    no += 1
    
print("final processed no = ", no)

In [ ]:
print("display processed val data for checking")

from IPython.display import display_png

no = 0
for i in range(2):
    print("processed no = ", no)
    val_image = Image.fromarray(np.uint8(val_image_array_list[i]), mode="RGB")
    display_png(val_image)
    grth_image = Image.fromarray(np.uint8(val_grth_image_array_list[i]), mode="P")
    grth_image.putpalette(palette)
    display_png(grth_image)
    no += 1
    
print("final processed no = ", no)

In [ ]:
train_input_data = np.array(train_image_array_list, dtype=np.float32)
train_input_data = train_input_data / np.float32(255.0)
train_input_data = train_input_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 3))

train_grth_data = np.array(train_grth_image_array_list)
train_grth_data = train_grth_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 1))

print("train_input_data.shape = ", train_input_data.shape)
print("train_grth__data.shape = ", train_grth_data.shape)
print("train_input_data and train_grth_data have been transformed successfully")

In [ ]:
val_input_data = np.array(val_image_array_list, dtype=np.float32)
val_input_data = val_input_data / np.float32(255.0)
val_input_data = val_input_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 3))

val_grth_data = np.array(val_grth_image_array_list)
val_grth_data = val_grth_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 1))

print("val_input_data.shape = ", val_input_data.shape)
print("val_grth__data.shape = ", val_grth_data.shape)
print("val_input_data and val_grth_data have been transformed successfully")

In [ ]:
def crop_size(tps_tensor, tgt_tensor):
    h_tps = tps_tensor.shape[1]
    w_tps = tps_tensor.shape[2]
    h_tgt = tgt_tensor.shape[1]
    w_tgt = tgt_tensor.shape[2]
    h_crop = (h_tps - h_tgt) // 2
    w_crop = (w_tps - w_tgt) // 2
    
    h = h_tgt + h_crop + h_crop
    assert h == h_tps, "h == h_tps does not hold"

    w = w_tgt + w_crop + w_crop
    assert w == w_tps, "w == w_tps does not hold"

    return h_crop, w_crop

In [ ]:
input = Input(shape=(RESIZE_HEIGHT, RESIZE_WIDTH, 3))

# AIFE4_CS2 Encoder
# Filter 1
e = Conv2D(FILT_NO1, FILT_SIZE, 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(AL2_VALUE1))(input)
e = BatchNormalization()(e)
    
# Filter 2
e = Conv2D(FILT_NO2, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(AL2_VALUE2))(e)
e = BatchNormalization()(e)

# Filter 3
e = Conv2D(FILT_NO3, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(AL2_VALUE3))(e)
e = BatchNormalization()(e)

# Filter 4, output is bottleneck
e = Conv2D(FILT_NO4, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(AL2_VALUE4))(e)
bottleneck = BatchNormalization()(e)

encoder = Model(inputs=[input], outputs=[bottleneck])
encoder.load_weights(encoder_path)

#FCN8s BLOCK
# Block 1
c = Conv2D(64, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(bottleneck)
c = BatchNormalization()(c)
c = Conv2D(64, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
pool1 = MaxPooling2D((2, 2), padding='same')(c)
    
# Block 2
c = Conv2D(128, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(pool1)
c = Conv2D(128, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
pool2 = MaxPooling2D((2, 2), padding='same')(c)

# Block 3
c = Conv2D(256, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(pool2)
c = Conv2D(256, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
c = Conv2D(256, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
pool3 = MaxPooling2D((2, 2), padding='same')(c)

# Block 4
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(pool3)
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
pool4 = MaxPooling2D((2, 2), padding='same')(c)

# Block 5
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(pool4)
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
c = Conv2D(512, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(c)
pool5 = MaxPooling2D((2, 2), padding='same')(c)

# fully conv
f = Conv2D(4096, (7, 7), 1, activation='relu' , padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(pool5)
d = Dropout(0.5)(f)
f = Conv2D(4096, (1, 1), 1, activation='relu', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(d)
d = Dropout(0.5)(f)
score_fr = Conv2D(NO_CLASS, (1, 1), 1, kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(d)

#scorinag pool4
score_pool4 = Conv2D(NO_CLASS, (1, 1) , 1, kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(pool4)
score_pool4 = BatchNormalization()(score_pool4)

# 2 times upsamping and cropping score_fr layer
upscore2 = Conv2DTranspose(NO_CLASS, kernel_size=(4, 4), strides=(2, 2), padding='valid', use_bias=False,
        kernel_initializer=KNL_INITLZ, kernel_regularizer=regularizers.l2(L2_VALUE3))(score_fr)
h_crop, w_crop = crop_size(upscore2, score_pool4)
print("h_crop = ", h_crop)
print("w_crop = ", w_crop)
h = np.int32(h_crop)
w = np.int32(w_crop)
upscore2c = Cropping2D(cropping=((h, h), (w, w)))(upscore2)
#upscore2c = Cropping2D(cropping=((1, 1), (1, 1)))(upscore2)
upscore2c = BatchNormalization()(upscore2c)
print("upscore2.shape = ", upscore2.shape)
print("upscore2c.shape = ", upscore2c.shape)

# Add [score_pool4, upscore2]
fuse_pool4 = Add()([score_pool4, upscore2c])

#scorining pool3
score_pool3 = Conv2D(NO_CLASS, (1, 1), 1, kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(pool3)
score_pool3 = BatchNormalization()(score_pool3)

# 2 times upsampling and cropping fuse_pool4
upscore_pool4 = Conv2DTranspose(NO_CLASS, kernel_size=(4, 4), strides=(2, 2), padding='valid', use_bias=False,
        kernel_initializer=KNL_INITLZ, kernel_regularizer=regularizers.l2(L2_VALUE3))(fuse_pool4)
h_crop, w_crop = crop_size(upscore_pool4, score_pool3)
h = np.int32(h_crop)
w = np.int32(w_crop)
upscore_pool4c = Cropping2D(cropping=((h, h), (w, w)))(upscore_pool4)
#upscore_pool4c = Cropping2D(cropping=((, 1), (1, 1)))(upscore_pool4)
upscore_pool4c = BatchNormalization()(upscore_pool4c)
print("upscore_pool4.shape = ", upscore_pool4.shape)
print("upscore_pool4c.shape = ", upscore_pool4c.shape)

# Add [score_pool3, upscore_pool4]
fuse_pool3 = Add()([score_pool3, upscore_pool4c])

# 8 times upsampling fuse_pool3 and cropping upscore8 into the input size
upscore8 = Conv2DTranspose(NO_CLASS, kernel_size=(16, 16),  strides=(8, 8), padding='valid', use_bias=False,
        kernel_initializer=KNL_INITLZ, kernel_regularizer=regularizers.l2(L2_VALUE3))(fuse_pool3)
h_crop, w_crop = crop_size(upscore8, input)
h = np.int32(h_crop)
w = np.int32(w_crop)
upscore8c = Cropping2D(cropping=((h, h), (w, w)))(upscore8)
#upscore8c = Cropping2D(cropping=((4, 4), (4, 4)))(upscore8)
print("upscore8.shape = ", upscore8.shape)
print("upscore8c.shape = ", upscore8c.shape)

score= (Activation('softmax'))(upscore8c)
print("score.shape = ", score.shape)

model = Model(inputs=[input], outputs=[score])

In [ ]:
sgd = SGD(lr=LEARN_RATE, momentum=MOMENTUM, nesterov=True)
#adam = Adam(lr=LEARN_RATE, beta_1=BETA_1)

model.compile(
#    optimizer = adam,
    optimizer = sgd,
    loss='sparse_categorical_crossentropy',
    metrics = ['accuracy']
)


In [ ]:
model.summary()

In [ ]:
tsb=TensorBoard(log_dir='.logs')
c_weight = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
print("c_weight = ", c_weight)
modelcp = ModelCheckpoint(filepath = checkpoint_path,
                                  monitor='val_acc',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=True,
                                  mode='max',
                                  period=1)

history1 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = 0,
    epochs = EPOCHS1,     
    batch_size = BATCH_SIZE,
    shuffle = True,   
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history1.npy", history1.history)

In [ ]:
history2 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1,
    epochs = EPOCHS1+EPOCHS2,    
    batch_size = BATCH_SIZE, 
    shuffle = True,   
    class_weight = c_weight,
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history2.npy", history2.history)

In [ ]:
history3 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1+EPOCHS2,
    epochs = EPOCHS1+2*EPOCHS2,     
    batch_size = BATCH_SIZE, 
    shuffle = True,   
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history3.npy", history3.history)

In [ ]:
history4 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1+2*EPOCHS2,
    epochs = EPOCHS1+3*EPOCHS2,    
    batch_size = BATCH_SIZE, 
    shuffle = True,   
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history4.npy", history4.history)

In [ ]:
history5 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1+3*EPOCHS2,
    epochs = EPOCHS1+4*EPOCHS2,    
    batch_size = BATCH_SIZE, 
    shuffle = True,   
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history5.npy", history5.history)

In [ ]:
history6 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1+4*EPOCHS2,
    epochs = EPOCHS1+5*EPOCHS2,    
    batch_size = BATCH_SIZE, 
    shuffle = True,   
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history6.npy", history6.history)

In [ ]:
history7 = model.fit(
    train_input_data, 
    train_grth_data, 
    initial_epoch = EPOCHS1+5*EPOCHS2,
    epochs = EPOCHS1+6*EPOCHS2,     
    batch_size = BATCH_SIZE, 
    shuffle = True,  
    class_weight = c_weight,    
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_grth_data)
    )
np.save("history7.npy", history7.history)

In [ ]:
model.load_weights(checkpoint_path)


In [ ]:
from IPython.display import display_png

def display_image(input_image, predict_image):
    display_png(input_image)
    display_png(predict_image) 

In [ ]:
# train image predict and save predict image

grth_file_list = os.listdir(path_train_grth)
tno_data = len(train_input_data)
i = 0
for i in range(10):    
    input = train_input_data[i]
    input_bt = np.expand_dims(input, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    predict = np.argmax(predict, axis=2)
    train_predict_image = Image.fromarray(np.uint8(predict), mode="P")
    train_predict_image.putpalette(palette)
    filename = grth_file_list[i]
    length = len(filename)
    length1 = length - 19
    string = filename[0:length1]
    filename1 = string + "TrainPredict.png"
    train_predict_image_r = train_predict_image.resize((2048,1024), PIL.Image.ANTIALIAS)    
    train_predict_image_r.save(path_train_predict_folder + "/" + filename1)

    train_grth_image_array = train_grth_image_array_list[i]
    train_grth_image = Image.fromarray(np.uint8(train_grth_image_array), mode="P")
    train_grth_image.putpalette(palette)

    display_png(train_grth_image)
    display_png(train_predict_image)
    print("i = ", i)

print("processing has finished successfully")

In [ ]:
# val image predict and save predict image

grth_file_list = os.listdir(path_val_grth)
tno_data = len(val_input_data)
i = 0
for i in range(tno_data):    
    input = val_input_data[i]
    input_bt = np.expand_dims(input, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    predict = np.argmax(predict, axis=2)
    val_predict_image = Image.fromarray(np.uint8(predict), mode="P")
    val_predict_image.putpalette(palette)
    filename = grth_file_list[i]
    length = len(filename)
    length1 = length - 19
    string = filename[0:length1]
    filename1 = string + "ValPredict.png"
    val_predict_image_r = val_predict_image.resize((2048,1024), PIL.Image.ANTIALIAS)    
    val_predict_image_r.save(path_val_predict_folder + "/" + filename1)

    val_grth_image_array = val_grth_image_array_list[i]
    val_grth_image = Image.fromarray(np.uint8(val_grth_image_array), mode="P")
    val_grth_image.putpalette(palette)

    if i%250 == 0:
        display_png(val_grth_image)
        display_png(val_predict_image)
        print("i = ", i)

print("i = ", i)
print("processing has finished successfully")

In [ ]:
train_loss, train_acc = model.evaluate(train_input_data, train_grth_data)
print("Restored model, train_loss: {:.4f}".format(train_loss))
print("Restored model, train_acc: {:.4f}".format(train_acc))


In [ ]:
val_loss, val_acc = model.evaluate(val_input_data, val_grth_data)
print("Restored model, val_loss: {:.4f}".format(val_loss))
print("Restored model, val_acc: {:.4f}".format(val_acc))
